In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"vaish414","key":"15f8c49438ce7260fe9e4194b7812137"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d prosperchuks/amazonsatelliteimages

Dataset URL: https://www.kaggle.com/datasets/prosperchuks/amazonsatelliteimages
License(s): unknown
 98% 1.47G/1.50G [00:19<00:00, 71.2MB/s]
100% 1.50G/1.50G [00:19<00:00, 80.5MB/s]


In [ ]:
!unzip amazonsatelliteimages.zip

Streaming output truncated to the last 5000 lines.
  inflating: train-jpg/train-jpg/train_55.jpg  
  inflating: train-jpg/train-jpg/train_550.jpg  
  inflating: train-jpg/train-jpg/train_5500.jpg  
  inflating: train-jpg/train-jpg/train_5501.jpg  
  inflating: train-jpg/train-jpg/train_5502.jpg  
  inflating: train-jpg/train-jpg/train_5503.jpg  
  inflating: train-jpg/train-jpg/train_5504.jpg  
  inflating: train-jpg/train-jpg/train_5505.jpg  
  inflating: train-jpg/train-jpg/train_5506.jpg  
  inflating: train-jpg/train-jpg/train_5507.jpg  
  inflating: train-jpg/train-jpg/train_5508.jpg  
  inflating: train-jpg/train-jpg/train_5509.jpg  
  inflating: train-jpg/train-jpg/train_551.jpg  
  inflating: train-jpg/train-jpg/train_5510.jpg  
  inflating: train-jpg/train-jpg/train_5511.jpg  
  inflating: train-jpg/train-jpg/train_5512.jpg  
  inflating: train-jpg/train-jpg/train_5513.jpg  
  inflating: train-jpg/train-jpg/train_5514.jpg  
  inflating: train-jpg/train-jpg/train_5515.jpg  
  i

In [ ]:
!kaggle datasets download -d abhisheksjha/time-series-air-quality-data-of-india-2010-2023

Dataset URL: https://www.kaggle.com/datasets/abhisheksjha/time-series-air-quality-data-of-india-2010-2023
License(s): CC-BY-NC-SA-4.0
100% 463M/464M [00:05<00:00, 115MB/s]
100% 464M/464M [00:05<00:00, 90.6MB/s]


In [ ]:
!unzip time-series-air-quality-data-of-india-2010-2023.zip -d air_quality_data

Archive:  time-series-air-quality-data-of-india-2010-2023.zip
  inflating: air_quality_data/AP001.csv  
  inflating: air_quality_data/AP002.csv  
  inflating: air_quality_data/AP003.csv  
  inflating: air_quality_data/AP004.csv  
  inflating: air_quality_data/AP005.csv  
  inflating: air_quality_data/AP006.csv  
  inflating: air_quality_data/AP007.csv  
  inflating: air_quality_data/AP008.csv  
  inflating: air_quality_data/AP009.csv  
  inflating: air_quality_data/AP010.csv  
  inflating: air_quality_data/AR001.csv  
  inflating: air_quality_data/AS001.csv  
  inflating: air_quality_data/AS002.csv  
  inflating: air_quality_data/AS003.csv  
  inflating: air_quality_data/AS004.csv  
  inflating: air_quality_data/AS005.csv  
  inflating: air_quality_data/AS006.csv  
  inflating: air_quality_data/AS007.csv  
  inflating: air_quality_data/AS008.csv  
  inflating: air_quality_data/AS009.csv  
  inflating: air_quality_data/BR001.csv  
  inflating: air_quality_data/BR002.csv  
  inflating: a

In [ ]:
import pandas as pd
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split

# Define the paths to your satellite images
base_dir = '/content'
paths = {
    'train_jpg': os.path.join(base_dir, 'train-jpg', 'train-jpg')
}

# Load labels
labels_df = pd.read_csv(os.path.join(base_dir, 'train_v2.csv', 'train_v2.csv'))

# Create binary labels for deforestation
labels_df['deforestation'] = labels_df['tags'].apply(lambda x: 1 if 'habitation' in x or 'slash_burn' in x else 0)

# Add file extensions to image filenames
labels_df['image_name'] = labels_df['image_name'].apply(lambda x: f"{x}.jpg")

# Split dataset into training and validation sets
train_df, val_df = train_test_split(labels_df, test_size=0.2, random_state=42)

# Function to load images from a directory
def load_images_from_df(df, directory):
    images = []
    labels = []
    for _, row in df.iterrows():
        img_path = os.path.join(directory, row['image_name'])
        if os.path.exists(img_path):
            img = Image.open(img_path).convert('RGB')
            img = img.resize((128, 128))  # Resize images to 128x128
            img = np.array(img)
            images.append(img)
            labels.append(row['deforestation'])
    return np.array(images), np.array(labels)

# Load images for training and validation
X_train, y_train = load_images_from_df(train_df, paths['train_jpg'])
X_val, y_val = load_images_from_df(val_df, paths['train_jpg'])

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input

# Define the satellite image model
image_input = Input(shape=(128, 128, 3))
x = Conv2D(32, (3, 3), activation='relu')(image_input)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
satellite_output = Dense(1, activation='sigmoid')(x)  # Use sigmoid for binary classification

satellite_model = Model(inputs=image_input, outputs=satellite_output)

# Compile the model
satellite_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history_satellite = satellite_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=25,
    batch_size=32
)

# Evaluate the model on the validation set
val_loss_satellite, val_acc_satellite = satellite_model.evaluate(X_val, y_val)
print(f'Validation Accuracy for Satellite Imagery Model: {val_acc_satellite:.2f}')

Epoch 1/25
1012/1012 [==============================] - 1375s 1s/step - loss: 0.4216 - accuracy: 0.9036 - val_loss: 0.3371 - val_accuracy: 0.9039
Epoch 2/25
1012/1012 [==============================] - 1288s 1s/step - loss: 0.3034 - accuracy: 0.9053 - val_loss: 0.2656 - val_accuracy: 0.9039
Epoch 3/25
1012/1012 [==============================] - 1317s 1s/step - loss: 0.2613 - accuracy: 0.9058 - val_loss: 0.2373 - val_accuracy: 0.9039
Epoch 4/25
1012/1012 [==============================] - 1314s 1s/step - loss: 0.2634 - accuracy: 0.9057 - val_loss: 0.2616 - val_accuracy: 0.9039
Epoch 5/25
1012/1012 [==============================] - 1256s 1s/step - loss: 0.2559 - accuracy: 0.9063 - val_loss: 0.2458 - val_accuracy: 0.9039
Epoch 6/25
1012/1012 [==============================] - 1301s 1s/step - loss: 0.2967 - accuracy: 0.9058 - val_loss: 0.3334 - val_accuracy: 0.9039
Epoch 7/25
1012/1012 [==============================] - 1255s 1s/step - loss: 0.3185 - accuracy: 0.9058 - val_loss: 0.3174 -

In [ ]:
# Save the model
model.save('deforestation_detection_model.h5')

In [ ]:
from tensorflow.keras.models import load_model

# Load the model
model = load_model('deforestation_detection_model.h5')

In [ ]:
# Load air quality data
air_quality_data_dir = '/content/air_quality_data'
air_quality_files = [os.path.join(air_quality_data_dir, file) for file in os.listdir(air_quality_data_dir) if file.endswith('.csv')]

# Combine all air quality data files
air_quality_dfs = [pd.read_csv(file) for file in air_quality_files]
combined_air_quality_df = pd.concat(air_quality_dfs, ignore_index=True)

# Handle missing values by dropping rows with NaN values
combined_air_quality_df.dropna(inplace=True)

# Generate predictions from the satellite image model
satellite_predictions = satellite_model.predict(X_train).flatten()

# Use the predictions as an additional feature in the air quality data
combined_air_quality_df['satellite_deforestation'] = satellite_predictions[:len(combined_air_quality_df)]

# Features and target for the air quality model
X_air_quality = combined_air_quality_df[['PM2.5 (ug/m3)', 'PM10 (ug/m3)', 'NO (ug/m3)', 'NO2 (ug/m3)', 'NOx (ppb)', 'NH3 (ug/m3)', 'SO2 (ug/m3)', 'CO (mg/m3)', 'satellite_deforestation']].values
y_air_quality = combined_air_quality_df['satellite_deforestation'].values  # Use deforestation prediction as the target

# Split the data into training and validation sets
X_train_air_quality, X_val_air_quality, y_train_air_quality, y_val_air_quality = train_test_split(X_air_quality, y_air_quality, test_size=0.2, random_state=42)

In [ ]:
from tensorflow.keras.layers import Input, Dense, Dropout

# Define the air quality model
air_quality_input = Input(shape=(X_train_air_quality.shape[1],))
x = Dense(64, activation='relu')(air_quality_input)
x = Dropout(0.5)(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.5)(x)
air_quality_output = Dense(1, activation='sigmoid')(x)  # Use sigmoid for binary classification

air_quality_model = Model(inputs=air_quality_input, outputs=air_quality_output)

# Compile the model
air_quality_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history_air_quality = air_quality_model.fit(
    X_train_air_quality, y_train_air_quality,
    validation_data=(X_val_air_quality, y_val_air_quality),
    epochs=25,
    batch_size=32
)

# Evaluate the model on the validation set
val_loss_air_quality, val_acc_air_quality = air_quality_model.evaluate(X_val_air_quality, y_val_air_quality)
print(f'Validation Accuracy for Air Quality Model: {val_acc_air_quality:.2f}')